In [ ]:
import urllib.request
from tkinter import *
from tkinter.ttk import *
from tkinter import messagebox
from gensim.summarization import summarize
from langdetect import detect
from googletrans import Translator
from bs4 import BeautifulSoup


class Summarizer(Frame):
    sampleText = '''From Tue planet is the fourth planet after the Sun and the solar system's \
second smallest planet Mercury. In English, Tue planet maintains the name of the Roman god of war, \
and is often referred to as the 'Red Planet'. The latter refers to the effect of \
the prevailing iron oxide on Tue surface, which gives it a red color between the celestial \
objects visible to the naked eye. [1 a] Tue is a terrestrial planet with a \
thin atmosphere, surface characteristics reminded recalled both craters influence \
of the moon and valleys, deserts and the Earth's polar ice caps.'''

    langKeyMap = {
        'af': 'Afrikaans',
        'ar': 'Arabic',
        'bg': 'Bulgarian',
        'bn': 'Bengali',
        'ca': 'Catalan',
        'cs': 'Czech',
        'cy': 'Welsh',
        'da': 'Danish',
        'de': 'German',
        'el': 'Greek',
        'en': 'English',
        'es': 'Spanish',
        'et': 'Estonian',
        'fa': 'Persian',
        'fi': 'Finnish',
        'fr': 'French',
        'gu': 'Gujarati',
        'he': 'Hebrew',
        'hi': 'Hindi',
        'hr': 'Croatian',
        'hu': 'Hungarian',
        'id': 'Indonesian',
        'it': 'Italian',
        'ja': 'Japanese',
        'kn': 'Kannada',
        'ko': 'Korean',
        'lt': 'Lithuanian',
        'lv': 'Latvian',
        'mk': 'Macedonian',
        'ml': 'Malayalam',
        'mr': 'Marathi',
        'ne': 'Nepali',
        'nl': 'Dutch',
        'no': 'Norwegian',
        'pa': 'Punjabi',
        'pl': 'Polish',
        'pt': 'Portuguese',
        'ro': 'Romanian',
        'ru': 'Russian',
        'sk': 'Slovak',
        'sl': 'Slovenian',
        'so': 'Somali',
        'sq': 'Albanian',
        'sv': 'Swedish',
        'sw': 'Swahili',
        'ta': 'Tamil',
        'te': 'Telugu',
        'th': 'Thai',
        'tl': 'Tagalog',
        'tr': 'Turkish',
        'uk': 'Ukrainian',
        'ur': 'Urdu',
        'vi': 'Vietnamese',
        'zh-cn': 'Chinese'
    }

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.master.title("Web articale summarizer")
        self.pack(fill=BOTH, expand=True)

        self.columnconfigure(1, weight=1)
        self.columnconfigure(3, pad=7)
        self.rowconfigure(3, weight=1)
        self.rowconfigure(5, pad=7)

        Label(self, text="Web articale summarizer") \
            .grid(sticky=W, row=0, column=0, pady=4, padx=5)

        Label(self, text="Select Language for transalation") \
            .grid(sticky=W, row=0, column=2, columnspan=2, pady=4, padx=5)

        self.lang = StringVar()

        self.lang.set('English')  # set the default option

        langs = list(self.langKeyMap.values())
        langs.sort()
        OptionMenu(
            self,
            self.lang,
            *langs
        ).grid(row=0, column=4)

        Label(self, text="Enter Url")\
            .grid(row=1, column=0, sticky=W, pady=4, padx=5)

        self.urlTextEntry = Entry(self)
        self.urlTextEntry.grid(row=1, column=1, columnspan=2, rowspan=1,
                               padx=5, sticky=E + W + S + N)

        Button(self, text="Fetch", command=self.readUrl) \
            .grid(row=1, column=4)

        self.textBox = Text(self, )
        self.textBox.grid(row=2, column=0, columnspan=3,
                          rowspan=10, padx=5, pady=15, sticky=E + W + S + N)
        self.textBox.insert('1.0', self.sampleText)

        Button(self, text="Translate", command=self.translate) \
            .grid(row=2, column=4, pady=15, sticky=N+S)
        Button(self, text="Summarize", command=self.summarize) \
            .grid(row=3, column=4, pady=4, sticky=N+S)
        Button(self, text="Identify", command=self.identify) \
            .grid(row=4, column=4, sticky=N + S, pady=15)

        self.titleText = StringVar()
        self.titleText.set("Pending Operation...")

        Label(self, textvariable=self.titleText) \
            .grid(sticky=W, row=13, column=0, pady=5, padx=5)
        Button(self, text="Clear", command=self.clear)\
            .grid(row=13, column=4, pady=5)

    def readUrl(self):
        inputURL = self.urlTextEntry.get()
        try:
            with urllib.request.urlopen(inputURL) as url:
                s = url.read()
                soup = BeautifulSoup(s, features='html.parser')
                fh = open('demo1.txt', "w+", encoding = "utf-8")
                for i in soup.select('p'):
                    #if(count == 5):
                     #   break;
                    #else:
                    #type(i.text)
                    fh.write(i.text)
                    #print(i.text)
                    #count = count + 1
                fh.close()
                fh = open('demo1.txt', "r", encoding = "utf-8")
                t1 = fh.read()
                t1 = re.sub("[\[].*?[\]]", "", t1)
                t1 = re.sub(' {[^}]*}','',t1)
                self.setText(t1,"Extracted from '" + inputURL + "'")
                self.urlTextEntry.delete(0, END)
        except Exception as e:
            messagebox.showerror(
                "Failed to fetch " + inputURL,
                "Reason: " + str(e)
            )

    def identify(self):
        inputText = self.textBox.get('1.0', END)

        pretext = "Given text is in: "

        try:
            lang = detect(inputText)
            if (lang in self.langKeyMap.keys()):
                lang = self.langKeyMap[lang]
                messagebox.showinfo("Identified language", pretext + lang)
                self.lang.set(lang)
            else:
                messagebox.showerror("Identified language", "Unknown language")

        except Exception as e:
            messagebox.showerror(
                "Failed", "Reason: " + str(e))

    def summarize(self):
        inputText = self.textBox.get('1.0', END)
        try:
            summary = summarize(inputText)
            self.setText(summary, "Summarized Text")
        except Exception as e:
            messagebox.showerror(
                "Can not summarize input text", "Reason: " + str(e))

    def setText(self, text, title=None):
        self.textBox.delete('1.0', END)
        self.textBox.insert('1.0', text)
        if not title is None:
            self.titleText.set(title)

    def translate(self):
        text = self.textBox.get('1.0', END)
        translator = Translator()
        try:
            lang = detect(text)

            if (not lang in self.langKeyMap.keys()):
                messagebox.showerror("Identified language", "Unknown language")
                return

            src = self.langKeyMap[lang].strip().lower()
            dest = self.lang.get().strip().lower()

            translated = translator.translate(
                text,
                src=src,
                dest=dest
            )

            toCopy = messagebox.askquestion(
                "Translated from " + src + " to " + dest,
                "Copy translated text to inputbox?\n\nTranslated text:\n" +
                translated.text
            )

            if toCopy == "yes":
                self.setText(
                    translated.text,
                    "Translated from " + src + " to " + dest
                )
        except Exception as e:
            messagebox.showerror(
                "Can not translate input text", "Reason: '" + str(e) + "'")

    def clear(self):
        self.titleText.set("Cleared")
        self.textBox.delete('1.0', END)
        self.urlTextEntry.delete('0', END)


def main():

    root = Tk()
    root.geometry("850x600+300+300")
    Summarizer()
    root.mainloop()


if __name__ == '__main__':
    main()


In [2]:
import urllib.request
from tkinter import *
from tkinter.ttk import *
from tkinter import messagebox
from gensim.summarization import summarize
from langdetect import detect
from googletrans import Translator
from bs4 import BeautifulSoup


class Summarizer(Frame):
    sampleText = '''From Tue planet is the fourth planet after the Sun and the solar system's \
second smallest planet Mercury. In English, Tue planet maintains the name of the Roman god of war, \
and is often referred to as the 'Red Planet'. The latter refers to the effect of \
the prevailing iron oxide on Tue surface, which gives it a red color between the celestial \
objects visible to the naked eye. [1 a] Tue is a terrestrial planet with a \
thin atmosphere, surface characteristics reminded recalled both craters influence \
of the moon and valleys, deserts and the Earth's polar ice caps.'''

    langKeyMap = {
        'af': 'Afrikaans',
        'ar': 'Arabic',
        'bg': 'Bulgarian',
        'bn': 'Bengali',
        'ca': 'Catalan',
        'cs': 'Czech',
        'cy': 'Welsh',
        'da': 'Danish',
        'de': 'German',
        'el': 'Greek',
        'en': 'English',
        'es': 'Spanish',
        'et': 'Estonian',
        'fa': 'Persian',
        'fi': 'Finnish',
        'fr': 'French',
        'gu': 'Gujarati',
        'he': 'Hebrew',
        'hi': 'Hindi',
        'hr': 'Croatian',
        'hu': 'Hungarian',
        'id': 'Indonesian',
        'it': 'Italian',
        'ja': 'Japanese',
        'kn': 'Kannada',
        'ko': 'Korean',
        'lt': 'Lithuanian',
        'lv': 'Latvian',
        'mk': 'Macedonian',
        'ml': 'Malayalam',
        'mr': 'Marathi',
        'ne': 'Nepali',
        'nl': 'Dutch',
        'no': 'Norwegian',
        'pa': 'Punjabi',
        'pl': 'Polish',
        'pt': 'Portuguese',
        'ro': 'Romanian',
        'ru': 'Russian',
        'sk': 'Slovak',
        'sl': 'Slovenian',
        'so': 'Somali',
        'sq': 'Albanian',
        'sv': 'Swedish',
        'sw': 'Swahili',
        'ta': 'Tamil',
        'te': 'Telugu',
        'th': 'Thai',
        'tl': 'Tagalog',
        'tr': 'Turkish',
        'uk': 'Ukrainian',
        'ur': 'Urdu',
        'vi': 'Vietnamese',
        'zh-cn': 'Chinese'
    }

    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.master.title("Web articale summarizer")
        self.pack(fill=BOTH, expand=True)

        self.columnconfigure(1, weight=1)
        self.columnconfigure(3, pad=7)
        self.rowconfigure(3, weight=1)
        self.rowconfigure(5, pad=7)

        Label(self, text="Web articale summarizer") \
            .grid(sticky=W, row=0, column=0, pady=4, padx=5)

        Label(self, text="Select Language for transalation") \
            .grid(sticky=W, row=0, column=2, columnspan=2, pady=4, padx=5)

        self.lang = StringVar()

        self.lang.set('English')  # set the default option

        langs = list(self.langKeyMap.values())
        langs.sort()
        OptionMenu(
            self,
            self.lang,
            *langs
        ).grid(row=0, column=4)

        Label(self, text="Enter Url")\
            .grid(row=1, column=0, sticky=W, pady=4, padx=5)

        self.urlTextEntry = Entry(self)
        self.urlTextEntry.grid(row=1, column=1, columnspan=2, rowspan=1,
                               padx=5, sticky=E + W + S + N)

        Button(self, text="Fetch", command=self.readUrl) \
            .grid(row=1, column=4)

        self.textBox = Text(self, )
        self.textBox.grid(row=2, column=0, columnspan=3,
                          rowspan=10, padx=5, pady=15, sticky=E + W + S + N)
        self.textBox.insert('1.0', self.sampleText)

        Button(self, text="Translate", command=self.translate) \
            .grid(row=2, column=4, pady=15, sticky=N+S)
        Button(self, text="Summarize", command=self.summarize) \
            .grid(row=3, column=4, pady=4, sticky=N+S)
        Button(self, text="Identify", command=self.identify) \
            .grid(row=4, column=4, sticky=N + S, pady=15)

        self.titleText = StringVar()
        self.titleText.set("Pending Operation...")

        Label(self, textvariable=self.titleText) \
            .grid(sticky=W, row=13, column=0, pady=5, padx=5)
        Button(self, text="Clear", command=self.clear)\
            .grid(row=13, column=4, pady=5)

    def readUrl(self):
        inputURL = self.urlTextEntry.get()
        try:
            with urllib.request.urlopen(inputURL) as url:
                s = url.read()
                soup = BeautifulSoup(s, features="lxml")
                self.setText(soup.body.get_text(),
                             "Extracted from '" + inputURL + "'")
                self.urlTextEntry.delete(0, END)
        except Exception as e:
            messagebox.showerror(
                "Failed to fetch " + inputURL,
                "Reason: " + str(e)
            )

    def identify(self):
        inputText = self.textBox.get('1.0', END)

        pretext = "Given text is in: "

        try:
            lang = detect(inputText)
            if (lang in self.langKeyMap.keys()):
                lang = self.langKeyMap[lang]
                messagebox.showinfo("Identified language", pretext + lang)
                self.lang.set(lang)
            else:
                messagebox.showerror("Identified language", "Unknown language")

        except Exception as e:
            messagebox.showerror(
                "Failed", "Reason: " + str(e))

    def summarize(self):
        inputText = self.textBox.get('1.0', END)
        try:
            summary = summarize(inputText)
            self.setText(summary, "Summarized Text")
        except Exception as e:
            messagebox.showerror(
                "Can not summarize input text", "Reason: " + str(e))

    def setText(self, text, title=None):
        self.textBox.delete('1.0', END)
        self.textBox.insert('1.0', text)
        if not title is None:
            self.titleText.set(title)

    def translate(self):
        text = self.textBox.get('1.0', END)
        translator = Translator()
        try:
            lang = detect(text)

            if (not lang in self.langKeyMap.keys()):
                messagebox.showerror("Identified language", "Unknown language")
                return

            src = self.langKeyMap[lang].strip().lower()
            dest = self.lang.get().strip().lower()

            translated = translator.translate(
                text,
                src=src,
                dest=dest
            )

            toCopy = messagebox.askquestion(
                "Translated from " + src + " to " + dest,
                "Copy translated text to inputbox?\n\nTranslated text:\n" +
                translated.text
            )

            if toCopy == "yes":
                self.setText(
                    translated.text,
                    "Translated from " + src + " to " + dest
                )
        except Exception as e:
            messagebox.showerror(
                "Can not translate input text", "Reason: '" + str(e) + "'")

    def clear(self):
        self.titleText.set("Cleared")
        self.textBox.delete('1.0', END)
        self.urlTextEntry.delete('0', END)


def main():

    root = Tk()
    root.geometry("850x600+300+300")
    Summarizer()
    root.mainloop()


if __name__ == '__main__':
    main()
